# Exercise 01: Subquery
## Required data

In [1]:
%ls ../data/checking-logs.sqlite

../data/checking-logs.sqlite


## Imports

In [2]:
import pandas as pd
import sqlite3

## Connect to DB and get `checker` table info

In [3]:
db_connection = sqlite3.connect('../data/checking-logs.sqlite')

In [4]:
pd.read_sql(sql='PRAGMA table_info(checker);', con=db_connection)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


## Get first 10 rows

In [5]:
pd.read_sql(sql='SELECT * FROM checker LIMIT 10;', con=db_connection, index_col='index')

,status,success,timestamp,numTrials,labname,uid
index,,,,,,
0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4


## Count the rows from the pageviews table but only with users from the checker table with:
* `status = ’ready’`   (we do not want to analyze the logs that are in status checking)
* `numTrials = 1`   (we want to analyze only the first commits, because only they can tell us when a student started working on a lab)
* `labname`s should be from the list: `’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’`.   (Only they were active during the experiment)

In [6]:
sql_query="""
SELECT 
    COUNT(*)
AS
    cnt
FROM 
    pageviews
WHERE uid IN (
    SELECT 
        uid
    FROM
        checker
    WHERE
        uid LIKE 'user_%'
        AND status = 'ready'
        AND numTrials = 1
        AND labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
)
"""

checkers = pd.read_sql(sql=sql_query, con=db_connection)
checkers

,cnt
0,984


## Close the connection

In [7]:
db_connection.close()